In [ ]:
!pip3 install pandas
!pip3 install folium
!pip3 install dask
!pip3 install "dask[distributed]" --upgrade
!pip3 install aiohttp

In [8]:
import pandas as pd
import folium
import requests
import os
import dask
import glob
from dask.distributed import Client
import dask.dataframe as dd

In [9]:
df_station = pd.read_csv("./isd-history.csv")
df_station
open_station = df_station[df_station["END"] == 20220502]
len(open_station)

353

In [10]:
dtypes = {
'STATION':         int,
'DATE':           object,
'SOURCE':         object,
'LATITUDE':      float,
'LONGITUDE':     float,
'ELEVATION':     float,
'NAME':           object,
'REPORT_TYPE':    object,
'CALL_SIGN':      object,
'QUALITY_CONTROL':object,
'WND':            object,
'CIG':            object,
'VIS':            object,
'TMP':            object,
'DEW':            object,
'SLP':            object,
'AA1':            object,
'AT1':            object,
'AT2':            object,
'AT3':            object,
'AT4':            object,
'AT5':            object,
'AU1':            object,
'AW1':            object,
'OC1':            object,
'REM':            object,
'EQD':            object
}

In [ ]:
client = Client(n_workers=4)

In [ ]:
if not os.path.exists("./data"):
    os.mkdir("./data")

for year in range(2022, 2023):
    year = str(year)
    path = "./data/" + year

    if not os.path.exists(path):
        os.mkdir(path)

    for key, station_row in open_station.iterrows():
        wban = str(station_row.WBAN);

        while (len(wban) < 5):
            wban = '0' + wban

        station  = str(station_row.USAF) + wban

        if not os.path.exists(path + "/" + station + ".csv"):
            url = 'https://www.ncei.noaa.gov/data/global-hourly/access/' + year + '/' + station + '.csv'

            if (requests.get(url).status_code == 200):
                #df = pd.read_csv(url)
                ddf = dd.read_csv(url, dtype='object')
                ddf.to_csv(path + "/" + station + ".csv", index=True, single_file = True)
                #df = pd.read_csv(url).to_csv(path + "/" + station + ".csv", index=True)
                #sleep(1)

In [12]:
dfs = []

for year in range(2022, 2023):
    year = str(year)
    path = "./data/" + year

    for filename in glob.glob(path + "/*.csv"):
        dfs.append(pd.read_csv(filename))

big_frame = pd.concat(dfs)
big_frame

,Unnamed: 0,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,...,RH3,WA1,AT7,ED1,GD4,MW3,MW4,GJ1,AU4,AW5
0,0,72211454901,2022-01-01T00:15:00,4,45.15917,-93.84333,294.7,"BUFFALO MUNICIPAL AIRPORT, MN US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,72211454901,2022-01-01T00:35:00,4,45.15917,-93.84333,294.7,"BUFFALO MUNICIPAL AIRPORT, MN US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,72211454901,2022-01-01T00:55:00,4,45.15917,-93.84333,294.7,"BUFFALO MUNICIPAL AIRPORT, MN US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,72211454901,2022-01-01T01:15:00,4,45.15917,-93.84333,294.7,"BUFFALO MUNICIPAL AIRPORT, MN US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,72211454901,2022-01-01T01:35:00,4,45.15917,-93.84333,294.7,"BUFFALO MUNICIPAL AIRPORT, MN US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8650,8650,72028203736,2022-05-02T06:35:00,6,36.01833,-75.67139,4.0,"KILL DEVIL HILLS FIRST FLIGHT AIRPORT, NC US",FM-15,KFFA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8651,8651,72028203736,2022-05-02T06:55:00,6,36.01833,-75.67139,4.0,"KILL DEVIL HILLS FIRST FLIGHT AIRPORT, NC US",FM-15,KFFA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8652,8652,72028203736,2022-05-02T07:15:00,6,36.01833,-75.67139,4.0,"KILL DEVIL HILLS FIRST FLIGHT AIRPORT, NC US",FM-15,KFFA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8653,8653,72028203736,2022-05-02T07:35:00,6,36.01833,-75.67139,4.0,"KILL DEVIL HILLS FIRST FLIGHT AIRPORT, NC US",FM-15,KFFA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Clean dataframe

In [ ]:
big_frame['TMP_control'] = big_frame['TMP'].str.split(pat=',', expand=True)[1]
big_frame['TMP_test'] = big_frame['TMP'].str.split(pat=',', expand=True)[0]
big_frame['TMP_sign'] = big_frame['TMP'].astype(str).str[0]
big_frame['TMP_final'] = (big_frame['TMP_test'].astype(str).str[1:4] + "." + big_frame['TMP_test'].astype(str).str[4:]).astype(float)
big_frame['DATE'] = pd.to_datetime(big_frame['DATE'], format="%Y-%m-%dT%H:%M:%S")
big_frame = big_frame[big_frame['TMP_control'] == "1"]

In [14]:
big_frame['TMP_sign'].replace({"+": 1, "-": -1}, inplace=True)
big_frame['TMP_final'] = big_frame['TMP_sign'] * big_frame['TMP_final']
big_frame

/var/folders/mb/dct658p12cs6fnf1h3084gph0000gn/T/ipykernel_9428/46208713.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_frame['TMP_sign'].replace({"+": 1, "-": -1}, inplace=True)
/var/folders/mb/dct658p12cs6fnf1h3084gph0000gn/T/ipykernel_9428/46208713.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_frame['TMP_final'] = big_frame['TMP_sign'] * big_frame['TMP_final']


,Unnamed: 0,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,...,GD4,MW3,MW4,GJ1,AU4,AW5,TMP_control,TMP_test,TMP_sign,TMP_final
0,0,72211454901,2022-01-01 00:15:00,4,45.15917,-93.84333,294.7,"BUFFALO MUNICIPAL AIRPORT, MN US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0150,-1,-15.0
1,1,72211454901,2022-01-01 00:35:00,4,45.15917,-93.84333,294.7,"BUFFALO MUNICIPAL AIRPORT, MN US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0150,-1,-15.0
2,2,72211454901,2022-01-01 00:55:00,4,45.15917,-93.84333,294.7,"BUFFALO MUNICIPAL AIRPORT, MN US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0150,-1,-15.0
3,3,72211454901,2022-01-01 01:15:00,4,45.15917,-93.84333,294.7,"BUFFALO MUNICIPAL AIRPORT, MN US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0150,-1,-15.0
4,4,72211454901,2022-01-01 01:35:00,4,45.15917,-93.84333,294.7,"BUFFALO MUNICIPAL AIRPORT, MN US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0160,-1,-16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8567,8567,72028203736,2022-05-01 03:35:00,4,36.01833,-75.67139,4.0,"KILL DEVIL HILLS FIRST FLIGHT AIRPORT, NC US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,1,+0131,1,13.1
8568,8568,72028203736,2022-05-01 03:55:00,4,36.01833,-75.67139,4.0,"KILL DEVIL HILLS FIRST FLIGHT AIRPORT, NC US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,1,+0130,1,13.0
8569,8569,72028203736,2022-05-01 04:15:00,4,36.01833,-75.67139,4.0,"KILL DEVIL HILLS FIRST FLIGHT AIRPORT, NC US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,1,+0130,1,13.0
8570,8570,72028203736,2022-05-01 04:35:00,4,36.01833,-75.67139,4.0,"KILL DEVIL HILLS FIRST FLIGHT AIRPORT, NC US",FM-15,99999,...,NaN,NaN,NaN,NaN,NaN,NaN,1,+0131,1,13.1


In [15]:
cleaned_frame = big_frame[["NAME", "TMP_final", "LATITUDE", "LONGITUDE", "DATE"]]
cleaned_frame

,NAME,TMP_final,LATITUDE,LONGITUDE,DATE
0,"BUFFALO MUNICIPAL AIRPORT, MN US",-15.0,45.15917,-93.84333,2022-01-01 00:15:00
1,"BUFFALO MUNICIPAL AIRPORT, MN US",-15.0,45.15917,-93.84333,2022-01-01 00:35:00
2,"BUFFALO MUNICIPAL AIRPORT, MN US",-15.0,45.15917,-93.84333,2022-01-01 00:55:00
3,"BUFFALO MUNICIPAL AIRPORT, MN US",-15.0,45.15917,-93.84333,2022-01-01 01:15:00
4,"BUFFALO MUNICIPAL AIRPORT, MN US",-16.0,45.15917,-93.84333,2022-01-01 01:35:00
...,...,...,...,...,...
8567,"KILL DEVIL HILLS FIRST FLIGHT AIRPORT, NC US",13.1,36.01833,-75.67139,2022-05-01 03:35:00
8568,"KILL DEVIL HILLS FIRST FLIGHT AIRPORT, NC US",13.0,36.01833,-75.67139,2022-05-01 03:55:00
8569,"KILL DEVIL HILLS FIRST FLIGHT AIRPORT, NC US",13.0,36.01833,-75.67139,2022-05-01 04:15:00
8570,"KILL DEVIL HILLS FIRST FLIGHT AIRPORT, NC US",13.1,36.01833,-75.67139,2022-05-01 04:35:00


# Statistiques globales de température (moy, min, max)

In [16]:
temp = cleaned_frame.groupby(by="NAME").agg({'TMP_final': ['mean', 'min', 'max'], "LATITUDE": "first", "LONGITUDE": "first"})
temp

TMP_final              \
                                                         mean   min   max   
NAME                                                                        
AHOSKIE TRI CO AIRPORT, NC US                       11.822093   0.0  20.2   
ALAMINOS CANYON, TX US                              21.320388  15.0  26.0   
ANDALUSIA OPP AIRPORT, AL US                        16.525000  13.9  21.1   
AUSTIN BERGSTROM INTERNATIONAL AIRPORT, TX US       14.773932  -5.6  32.8   
BAINBRIDGE DECATUR CO INDUSTRIAL AIR PARK AIRPO...  17.641228   3.6  26.1   
...                                                       ...   ...   ...   
WAHPETON HARRY STERN AIRPORT, ND US                 -5.295918 -29.0  10.0   
WHARTON REGIONAL AIRPORT, TX US                     18.773529   4.0  28.7   
WILLMAR MUNICIPAL AIRPORT JOHN L RICE FIELD, MN US  -1.309278 -25.0  14.0   
WINSTON SALEM REYNOLDS AIRPORT, NC US               13.566667   8.0  21.0   
ZEPHYRHILLS MUNICIPAL AIRPORT, FL US                20.015294   8.9  30.7   

                                                    LATITUDE LONGITUDE  
                                                       first     first  
NAME                                                                    
AHOSKIE TRI CO AIRPORT, NC US                       36.29750 -77.17083  
ALAMINOS CANYON, TX US                              26.93333 -94.68333  
ANDALUSIA OPP AIRPORT, AL US                        31.30614 -86.39018  
AUSTIN BERGSTROM INTERNATIONAL AIRPORT, TX US       30.18311 -97.67989  
BAINBRIDGE DECATUR CO INDUSTRIAL AIR PARK AIRPO...  30.98333 -84.63333  
...                                                      ...       ...  
WAHPETON HARRY STERN AIRPORT, ND US                 46.24417 -96.60722  
WHARTON REGIONAL AIRPORT, TX US                     29.26611 -96.00750  
WILLMAR MUNICIPAL AIRPORT JOHN L RICE FIELD, MN US  45.11700 -95.13300  
WINSTON SALEM REYNOLDS AIRPORT, NC US               36.13347 -80.22441  
ZEPHYRHILLS MUNICIPAL AIRPORT, FL US                28.22813 -82.15592  

[125 rows x 5 columns]

# Affichage Station (MAP)

In [17]:
m = folium.Map(location=[temp["LATITUDE"].mean(), temp["LONGITUDE"].mean()], zoom_start=5)

for i in range(0,len(temp)):
   folium.Marker(
      location=[temp.iloc[i]['LATITUDE'], temp.iloc[i]['LONGITUDE']],
      popup=folium.Popup(temp.iloc[i].to_frame().to_html(), max_width=450,min_height=400)
   ).add_to(m)
   
m

In [ ]:
#for year in range(2022, 2023):
#    year = str(year)
#    path = "./data/" + year
#
#    for key, station_row in open_station.iterrows():
#        wban = str(station_row.WBAN);
#
#        while (len(wban) < 5):
#            wban = '0' + wban
#
#        station  = str(station_row.USAF) + wban
#
#        if os.path.exists(path + "/" + station + ".csv"):
#            #df = pd.read_csv(path + "/" + station + ".csv")
#            #print(df.dtypes)
#            df = dd.read_csv(path + "/" + station + ".csv", dtype=dtypes)
#            
#            
#            #df = dd.read_csv(path + "/" + station + ".csv")
#            #df.head()


In [ ]:
#for year in range(2022, 2023):
#    year = str(year)
#    path = "./data/" + year

#    if not os.path.exists(path):
#        os.mkdir(path)

#    for key, station_row in open_station.iterrows():
#        wban = str(station_row.WBAN);

#        while (len(wban) < 5):
#            wban = '0' + wban

#        station  = str(station_row.USAF) + wban

#        if not os.path.exists(path + "/" + station + ".csv"):
#            url = 'https://www.ncei.noaa.gov/data/global-hourly/access/' + year + '/' + station + '.csv'

#            if (requests.get(url).status_code == 200):
#                pd.read_csv(url).to_csv(path + "/" + station + ".csv")

#            spark.sparkContext.addFile('https://www.ncei.noaa.gov/data/global-hourly/access/' + year + '/' + station + '.csv')

#            if (SparkFiles.get(station + ".csv")):
#                df = spark.read.csv("file://"+SparkFiles.get(station + ".csv"), header=True)

#                print("hdfs://namenode:9000/noaa/" + year + "/" + station + ".csv")
#                df.write.format("csv").save("hdfs://namenode:9000/noaa/" + year + "/" + station + ".csv")

#for year in range(2022, 2023):

#    for key, station_row in open_station.iterrows():
#        wban = str(station_row.WBAN);

#        while (len(wban) < 5):
#            wban = '0' + wban

#        station  = str(station_row.USAF) + wban
        
#        df = spark.read.csv("hdfs://namenode:9000/noaa/" + str(year) + "/" + station + ".csv", header=True)
#df = spark.read.csv("hdfs://namenode:9000/noaa/2022/72026853882.csv", header=True)
#df.take(5)